Building small core programs from [nsepython](https://github.com/aeron7/nsepython/blob/master/nsepython/rahu.py) and [nsepy](https://github.com/swapniljariwala/nsepy)   
- [ ] Get correct Volatility (from Standard Deviations)   



In [5]:
1.8/(620-610-1.8)
450000/50000*3
1.7/8

0.2125

In [ ]:
# Imports
import datetime

In [ ]:
# Get the most volatile symbol
import nsepython
df_ad = nsepython.nse_get_advances_declines().reset_index(drop=True)

df_ad.tail(10)[['symbol', 'lastPrice']]

In [6]:
symbol = df_ad.symbol.iloc[-2]
symbol

'INTELLECT'

In [7]:
# Inputs
# symbol = 'SBIN'
port = 3000
period = 365 # days
end = datetime.datetime.now().date()
start = end - datetime.timedelta(days=period)
sd_fence = 2

In [8]:
# Get equity history
from nsepy import get_history
df_equity = get_history(symbol=symbol, start=start, end=end)

In [9]:
# .cleanup
# ..remove gaps and % in column names and make lowercase
cols = {e: e.replace("%", "pct_")\
            .replace(" ", "_")\
            .lower() \
               for e in list(df_equity)}
    
# .. make lowercase fieldnames
df_equity.index.names = ['date']
df_equity = df_equity.rename(columns=cols)

In [10]:
# Compute historical standard deviations
df_equity = df_equity.assign(rollsd=df_equity.close.pct_change().expanding(1).std(ddof=0)*252**0.5)
hv = df_equity.rollsd.iloc[-1] # annual historical volatility

In [11]:
# Let us see what the IV calculated by IBKR is :)
# ....Note: API Historical data corresponds to the data in TWS charts. 
#     So to find the data in TWS you have to create a chart for the stock and set the "What To Show" parameter in the Chart Parameters to "Option Implied Volatility". 
#     The option IV at a time is calculated based on options which expire in the two months after that date. The data goes back pretty far, 5-10 years generally.
#  
from ib_insync import IB, Contract, Option, util, MarketOrder
import numpy as np
import logging

util.startLoop()

ib = IB()

In [12]:
# Prepare symbol for IB
ib_sym = symbol[:9]
ib_sym = ib_sym.replace('&', '')

if ib_sym == 'NIFTY': 
    ib_sym = 'NIFTY50'

secType = 'IND' if 'NIFTY' in ib_sym.upper() else 'STK'


In [13]:
try:

    ib.connect(port=3000)

    c = Contract(secType=secType, symbol=ib_sym, exchange='NSE')
    ib.qualifyContracts(c)

    bar_iv = ib.reqHistoricalData(c, endDateTime='', durationStr = '1 D', barSizeSetting='1 day', whatToShow='OPTION_IMPLIED_VOLATILITY', useRTH=True)
    ib_iv = bar_iv[-1].close

    bar_hv = ib.reqHistoricalData(c, endDateTime='', durationStr = '2 D', barSizeSetting='1 day', whatToShow='HISTORICAL_VOLATILITY', useRTH=True)
    
    ib_hv = bar_hv[-1].close

except Exception as e:
    
    logging.error(e)

    ib_hv = ib_iv = np.nan

ib.disconnect()

In [14]:
# ...establish an iv as the max
try:
    iv = max(hv, ib_iv, ib_hv)
except Exception as e:
    iv = hv

In [15]:
iv, ib_hv

(0.4576931853698657, 0.40479995)

In [16]:
# Get option chain
from nsepython import nse_optionchain_scrapper, nse_get_fno_lot_sizes
import pandas as pd
import pytz

scraped = nse_optionchain_scrapper(symbol)

raw_dict = scraped['records']['data']
dfs = [pd.DataFrame.from_dict(raw_dict[i]).transpose()[2:] 
            for i in range(len(raw_dict))]

df = pd.concat(dfs).drop(columns='identifier').rename_axis('right').reset_index()

float_cols = ['strikePrice', 'pchangeinOpenInterest', 'impliedVolatility', 
            'lastPrice', 'change', 'pChange', 'bidprice', 'askPrice' , 
            'underlyingValue' ]
int_cols = ['openInterest', 'changeinOpenInterest', 'totalTradedVolume', 
            'totalBuyQuantity', 'totalSellQuantity', 'bidQty', 'askQty']
df[float_cols] = df[float_cols].astype('float32')
df[int_cols] = df[int_cols].astype('int64')
df.loc[:, 'expiryDate'] = pd.to_datetime(df.expiryDate)

colmap = { 'underlying': 'symbol', 'expiryDate': 'expiry', 'strikePrice': 'strike',  
        'right': 'right','underlyingValue': 'undPrice', 'openInterest': 'oi',
        'changeinOpenInterest': 'oiChange', 'pchangeinOpenInterest': 'pChangeOI',
        'totalTradedVolume': 'volume', 'totalBuyQuantity': 'totalBuyQty', 
        'totalSellQuantity': 'totalSellQty', 'pChange': 'pChange', 
        'impliedVolatility': 'opt_iv',   'lastPrice': 'lastPrice', 'change': 'change',
        'bidQty': 'bidQty', 'bidprice': 'bid', 'askPrice': 'ask', 'askQty': 'askQty',
        }

df = df.rename(columns=colmap)[colmap.values()]
df = df.assign(lot=nse_get_fno_lot_sizes(symbol=symbol))
df = df.assign(opt_iv = df.opt_iv/100)
df = df.assign(right=df.right.str[:1])
df = df.sort_values(['expiry', 'right', 'strike'], 
                    ascending=[True, False, True]).reset_index(drop=True)


# ..get accurate dte
nse_tz = pytz.timezone('Asia/Kolkata')
now = datetime.datetime.now(tz=nse_tz).timestamp()
nse_tz_expiry = df.expiry.apply(lambda x: nse_tz.localize(datetime.datetime.combine(x.date(), datetime.time(18,0))).timestamp())
dte = (nse_tz_expiry.apply(datetime.datetime.fromtimestamp) - datetime.datetime.fromtimestamp(now)).apply(datetime.timedelta.total_seconds)/24/3600
df = df.assign(dte=dte)

In [17]:
# Identify target puts
df1 = df[(df.right == 'P') & (df.strike < df.undPrice) & (df.opt_iv > 0)]
df1 = df1.assign(tgt_put=df1.undPrice*(1-df1.opt_iv*sd_fence*(df1.dte/365)**0.5)) # using opt_iv

# df1 = df1.assign(tgt_put=df1.undPrice*(1-iv*sd_fence*(df1.dte/365)**0.5)) # using underlying iv

df_tgt = df1[df1.strike < df1.tgt_put]

In [23]:
df[df.right.eq('P') & df.dte.lt(25)]

,symbol,expiry,strike,right,undPrice,oi,oiChange,pChangeOI,volume,totalBuyQty,...,pChange,opt_iv,lastPrice,change,bidQty,bid,ask,askQty,lot,dte
0,INTELLECT,2022-12-29,330.0,P,427.5,3,0,0.000000,0,3750,...,0.000000,0.0000,0.000000,0.00,3750,0.100000,1.000000,2250,750,20.132999
1,INTELLECT,2022-12-29,340.0,P,427.5,0,0,0.000000,0,9750,...,0.000000,0.0000,0.000000,0.00,9750,0.100000,1.000000,3750,750,20.132999
2,INTELLECT,2022-12-29,350.0,P,427.5,0,0,0.000000,0,9750,...,0.000000,0.0000,0.000000,0.00,9750,0.100000,1.250000,3750,750,20.132999
3,INTELLECT,2022-12-29,360.0,P,427.5,0,0,0.000000,0,9750,...,0.000000,0.0000,0.000000,0.00,9750,0.100000,4.950000,37500,750,20.132999
4,INTELLECT,2022-12-29,370.0,P,427.5,0,0,0.000000,0,10500,...,0.000000,0.0000,0.000000,0.00,750,0.200000,5.050000,750,750,20.132999
5,INTELLECT,2022-12-29,380.0,P,427.5,17,7,70.000000,10,66750,...,-66.336632,0.3991,1.700000,-3.35,1500,2.050000,2.250000,3750,750,20.132999
6,INTELLECT,2022-12-29,390.0,P,427.5,0,0,0.000000,0,51000,...,0.000000,0.0000,0.000000,0.00,750,1.450000,5.450000,750,750,20.132999
7,INTELLECT,2022-12-29,400.0,P,427.5,267,58,27.751196,381,99000,...,311.538452,0.4087,5.350000,4.05,750,5.100000,5.250000,750,750,20.132999
8,INTELLECT,2022-12-29,410.0,P,427.5,9,-1,-10.000000,11,97500,...,128.169022,0.4065,8.100000,4.55,750,7.400000,7.900000,3750,750,20.132999
9,INTELLECT,2022-12-29,420.0,P,427.5,113,-6,-5.042017,144,116250,...,220.289856,0.3887,11.050000,7.60,750,10.850000,11.300000,750,750,20.132999


In [ ]:
# Get margins from IBKR

# ... build the contracts
optcons = [
    Option(s, e, k, r, "NSE")
    for s, e, k, r in zip(df_tgt.symbol, df_tgt.expiry.dt.date.apply(str).replace('-','', regex=True), df_tgt.strike, df_tgt.right)
]

with ib.connect(port=port) as my_ib:
    my_ib.qualifyContracts(*optcons)

df_tgt.insert(0, 'conId', [c.conId for c in optcons])

# ... build the orders
orders = [MarketOrder("SELL", qty) for qty in df_tgt.lot]

# ... get commissions and margins
import asyncio
async def get_margins(contracts: list, orders: list, timeout: float=2) -> pd.DataFrame:

    async def wifAsync(ct, o):
        wif = ib.whatIfOrderAsync(ct, o)
        try:
           res = await asyncio.wait_for(wif, timeout=timeout)
        except asyncio.TimeoutError:
           res = None
        return res
    
    wif_tasks = [asyncio.create_task(wifAsync(contract, order), name=contract.conId) for contract, order in zip(contracts, orders)]
    
    res = await asyncio.gather(*wif_tasks)
    
    margins = [{'margin': r.initMarginChange, 'commission': r.commission} for r in res if r]
    conIds = [c.conId for c in contracts]

    results = dict(zip(conIds, margins))

    df = pd.DataFrame(results).transpose()
    
    return df

with ib.connect(port=port) as my_ib:
    df_margin = ib.run(get_margins(optcons, orders))



In [ ]:
g = df1.sort_values(['expiry', 'strike']).groupby('expiry')
g.tail(1)

In [ ]:
# ... compute the profitablity
df_tgt = df_tgt.set_index('conId').join(df_margin.astype(float))
df_tgt = df_tgt.assign(rom=(df_tgt.ask*df_tgt.lot-df_tgt.commission)/df_tgt.margin*365/df_tgt.dte)

In [ ]:
cols = ['symbol', 'expiry', 'strike', 'right', 'undPrice', 'oi', 'volume', 'totalBuyQty', 'totalSellQty', 'opt_iv', 'lastPrice', 
        'bidQty', 'bid', 'ask', 'askQty', 'lot', 'dte', 'tgt_put', 'margin', 'rom']
df_tgt[cols].sort_values('rom', ascending=False)

In [ ]:
df_tgt

In [ ]:
from nsetools import Nse
nse = Nse()
code = 'SBIN22NOV'
# code = 'SBIN'
# getting stock quote
quote = nse.get_quote(code)
 
# getting applicable margin
value = quote['applicableMargin']
 
# printing applicable margin
print("Applicable Margin  : " + str(value))